In [1]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
Data_Dir ='/tmp/data'

MNIST = input_data.read_data_sets(Data_Dir, one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
MNIST

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f49a2cb1cc0>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f499c815cf8>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f499c815a90>)

In [5]:
#Set the learning parameters
learning_rate = 0.01
training_iteration = 30
mini_batch_size = 100

In [6]:
#Tensorflow graph input
x = tf.placeholder(tf.float32, shape = (None, 784)) #mnist image of shape 28*28=784
y = tf.placeholder(tf.float32, shape = (None, 10)) #0-9 digits recognition

In [7]:
#Set model weights
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
#Create NameScope to group model-related nodes
with tf.name_scope("Wx_b") as scope:
    #Construct a linear model
    model = tf.nn.softmax(tf.matmul(x,W) + b) #Softmax

In [9]:
#Add summary to collect data
w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

In [10]:
#NameScope to group cost-function related nodes
with tf.name_scope("cost_function") as scope:
    #Minimise the models errors using cross entropy
    #Cross-entropy
    cost_function = -tf.reduce_sum(y*tf.log(model))
    #Create a summary to collect data on the cost function
    tf.summary.scalar("cost_function", cost_function)

In [11]:
#Namescope to group Gradient-Descent related nodes
with tf.name_scope("Gradient_Descent") as scope:
    #Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [16]:
#Initialising the variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [13]:
#Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

In [14]:
MNIST.test.num_examples

10000

In [15]:
MNIST.train.num_examples

55000

In [18]:
total_batch = MNIST.train.num_examples//mini_batch_size
total_batch

550

In [17]:
#Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    #write summaries to the folder /home/ken/my_tensorflow
    FileWriter = tf.summary.FileWriter('/home/ken/venv', tf.get_default_graph())
    
    #Training cycle
    for iteration in range(training_iteration):
        avg_cost = 0
        total_batch = MNIST.train.num_examples//mini_batch_size
        for i in range(total_batch):
            batch_xs, batch_ys =MNIST.train.next_batch(mini_batch_size)
            sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys})

            #Compute the average loss
            avg_cost =0
            avg_cost += sess.run(cost_function, feed_dict={x:batch_xs, y:batch_ys})

            #Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            FileWriter.add_summary(summary_str)
            saver.save(sess, '/home/ken/venv/myModels/softmax_regr_mini_batch_gd.ckpt')
            
        display_step =2
        #Display logs per iteration
        if iteration % display_step ==0:
            print ("Iteration:",'%04d' % (iteration + 1), "cost=",'{:.9}'.format(avg_cost))

    print ('Tuning completed')

    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))

    #Calculate Accuracy
    accuracy =tf.reduce_mean(tf.cast(predictions, dtype=tf.float32))
    print ("Accuracy", accuracy.eval({x: MNIST.test.images, y: MNIST.test.labels}))

    

Iteration: 0001 cost= 14.3548841
Iteration: 0003 cost= 18.0803165
Iteration: 0005 cost= 20.5399876
Iteration: 0007 cost= 19.8984814
Iteration: 0009 cost= 19.849472
Iteration: 0011 cost= 5.78969383
Iteration: 0013 cost= 35.9071541
Iteration: 0015 cost= 22.3915367
Iteration: 0017 cost= 13.6197758
Iteration: 0019 cost= 16.4542599
Iteration: 0021 cost= 8.68281841
Iteration: 0023 cost= 9.38771725
Iteration: 0025 cost= 11.9201126
Iteration: 0027 cost= 46.079792
Iteration: 0029 cost= 18.0578194
Tuning completed
Accuracy 0.9191


In [18]:
FileWriter.close()